In [4]:
## odd even transposition sorting 

!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-q2sj38w4
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-q2sj38w4
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=e9408cfbf63a64d7d625ca114aadc5e4d3614037207c52c0e637207b35fdb572
  Stored in directory: /tmp/pip-ephem-wheel-cache-kclcshug/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [9]:
%reload_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__device__ void swap(int *a, int *b)
{
 int temp = *a;
 *a = *b;
 *b = temp;}
__global__ void oddEven(int *a, int n)
{
 int i = (blockDim.x * blockIdx.x) + threadIdx.x;
 if ((i % 2 == 1) && (i < n - 1))
 {
 if (a[i] > a[i + 1])
 {
 swap(&a[i], &a[i + 1]);
 }
 }
}
__global__ void evenOdd(int *a, int n)
{
 int i = (blockDim.x * blockIdx.x) + threadIdx.x;
 if ((i % 2 == 0) && (i < n - 1))
 {
 if (a[i] > a[i + 1])
 {
 swap(&a[i], &a[i + 1]);
 }
 }
}
void printVector(int *a, int n)
{
 for (int i = 0; i < n; i++)
 {
 printf("%d, ", a[i]);
 }
 printf("\n");
}
int main()
{
 int *d_a;
 int n = 10;
 int a[n] = {13, 10, 4, 7, 1, 4, 8, 0, 5, 6};
 printVector(a, n);
 cudaMalloc((void **)&d_a, n * sizeof(int));
 cudaMemcpy(d_a, a, n * sizeof(int), cudaMemcpyHostToDevice);
 for (int i = 0; i <= n / 2; i++)
 {
 oddEven<<<ceil(n / 256.0), 256>>>(d_a, n);
 evenOdd<<<ceil(n / 256.0), 256>>>(d_a, n);
 }
 cudaMemcpy(a, d_a, n * sizeof(int), cudaMemcpyDeviceToHost);
 printVector(a, n);
 cudaFree(d_a);
}